In [25]:
import sys
sys.path.append('/zfsauton3/home/bparia/hf_moe')

import data_loader
import pickle
import numpy as np
from tqdm import tqdm

In [64]:
dataset = 'm5'
with open(f'../data/{dataset}/data.pkl', 'rb') as fin:
    tree, ts_data, (feats, _, _) = pickle.load(fin)

leaf_mat = tree.leaf_matrix.T
num_leaf = np.sum(leaf_mat, axis=0, keepdims=True)
ts_data = ts_data / num_leaf

steps, nodes = ts_data.shape

ts_data = np.expand_dims(ts_data, axis=-1)
ts_data.shape

(1913, 3060, 1)

In [54]:
with open(f'../../DCRNN/data/sensor_graph/adj_{dataset}.pkl', 'wb') as f:
    pickle.dump([None, tree.levels, tree.adj_matrix], f, protocol=2)

In [55]:
hist_len = 28
pred_len = 7

tot_len = ts_data.shape[0]

val_windows = 5
test_windows = 5

In [65]:
data_path = f'../../DCRNN/data/{dataset}/train.npz'

num_data = tot_len - (val_windows + test_windows) * pred_len \
        - 2 * hist_len

train_x = []
train_y = []

for i in tqdm(range(num_data)):
    sub_ts = ts_data[i:i+hist_len+pred_len]
    sub_x = sub_ts[:hist_len]
    sub_y = sub_ts[hist_len:, :, :1]
    train_x.append(sub_x)
    train_y.append(sub_y)

train_x = np.array(train_x)
train_y = np.array(train_y)

print(train_x.shape)
print(train_y.shape)

np.savez_compressed(
    data_path,
    x=train_x,
    y=train_y
)

100%|██████████| 1787/1787 [00:00<00:00, 112256.01it/s]
(1787, 28, 3060, 1)
(1787, 7, 3060, 1)


In [66]:
data_path = f'../../DCRNN/data/{dataset}/val.npz'

start_idx = tot_len - (val_windows + test_windows) * pred_len - hist_len
end_idx = tot_len - (test_windows + 1) * pred_len - hist_len

val_x = []
val_y = []

for i in range(start_idx, end_idx+1, pred_len):
    sub_ts = ts_data[i:i+hist_len+pred_len]
    sub_x = sub_ts[:hist_len]
    sub_y = sub_ts[hist_len:, :, :1]
    val_x.append(sub_x)
    val_y.append(sub_y)

val_x = np.array(val_x)
val_y = np.array(val_y)

print(val_x.shape)
print(val_y.shape)

np.savez_compressed(
    data_path,
    x=val_x,
    y=val_y
)

(5, 28, 3060, 1)
(5, 7, 3060, 1)


In [67]:
data_path = f'../../DCRNN/data/{dataset}/test.npz'

start_idx = tot_len - test_windows * pred_len - hist_len
end_idx = tot_len - pred_len - hist_len
test_x = []
test_y = []

for i in range(start_idx, end_idx+1, pred_len):
    sub_ts = ts_data[i:i+hist_len+pred_len]
    sub_x = sub_ts[:hist_len]
    sub_y = sub_ts[hist_len:, :, :1]
    test_x.append(sub_x)
    test_y.append(sub_y)

test_x = np.array(test_x)
test_y = np.array(test_y)

print(test_x.shape)
print(test_y.shape)

np.savez_compressed(
    data_path,
    x=test_x,
    y=test_y
)

(5, 28, 3060, 1)
(5, 7, 3060, 1)
